**Importación de paquetes**

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

**Variables climáticas**

In [ ]:
Direccion_Viento = pd.read_csv('/content/1. Direccion Viento.csv')
Humedad = pd.read_csv('/content/2. Humedad.csv')
Precipitacion = pd.read_csv('/content/3. Precipitacion.csv') # Precipitación anual acumulada
Presion_Atmosferica = pd.read_csv('/content/4. Presion Atmosferica.csv')
Temperatura = pd.read_csv('/content/5. Temperatura.csv')
Temperatura_Maxima = pd.read_csv('/content/6. Temperatura Maxima.csv')
Temperatura_Minima = pd.read_csv('/content/7 .Temperatura Minima.csv')
Velocidad_Viento = pd.read_csv('/content/8. Velocidad Viento.csv')

In [ ]:
# Juntamos los datasets con un full outer Join, en las columnas año y municipio, posteriormente estandarizamos los labels y los nombres de los municipios
Variables_Climaticas = pd.merge(Direccion_Viento,Humedad, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas=Variables_Climaticas.rename(columns={"ValorObservado_x":"DIRECCION_VIENTO","ValorObservado_y":"HUMEDAD"})

Variables_Climaticas_1=pd.merge(Variables_Climaticas,Precipitacion, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_1=Variables_Climaticas_1.rename(columns={"ValorObservado":"PRECIPITACION"})

Variables_Climaticas_2=pd.merge(Variables_Climaticas_1,Presion_Atmosferica, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_2=Variables_Climaticas_2.rename(columns={"ValorObservado":"PRESION_ATMOSFERICA"})

Variables_Climaticas_3=pd.merge(Variables_Climaticas_2,Temperatura, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_3=Variables_Climaticas_3.rename(columns={"ValorObservado":"TEMPERATURA"})

Variables_Climaticas_4=pd.merge(Variables_Climaticas_3,Temperatura_Maxima, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_4=Variables_Climaticas_4.rename(columns={"ValorObservado":"TEMPERATURA_MAXIMA"})

Variables_Climaticas_5=pd.merge(Variables_Climaticas_4,Temperatura_Minima, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_5=Variables_Climaticas_5.rename(columns={"ValorObservado":"TEMPERATURA_MINIMA"})

Variables_Climaticas_Consolidado=pd.merge(Variables_Climaticas_5,Velocidad_Viento, on=["FechaObservacion","Departamento","Municipio"], how='outer')
Variables_Climaticas_Consolidado=Variables_Climaticas_Consolidado.rename(columns={"ValorObservado":"VELOCIDAD_VIENTO","FechaObservacion":"AÑO","Departamento":"DEPARTAMENTO","Municipio":"MUNICIPIO"})
Variables_Climaticas_Consolidado['MUNICIPIO']=Variables_Climaticas_Consolidado['MUNICIPIO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
Variables_Climaticas_Consolidado['DEPARTAMENTO']=Variables_Climaticas_Consolidado['DEPARTAMENTO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

Variables_Climaticas_Consolidado = Variables_Climaticas_Consolidado[Variables_Climaticas_Consolidado['AÑO'].between(2007, 2021)]
Variables_Climaticas_Consolidado

,DEPARTAMENTO,MUNICIPIO,AÑO,DIRECCION_VIENTO,HUMEDAD,PRECIPITACION,PRESION_ATMOSFERICA,TEMPERATURA,TEMPERATURA_MAXIMA,TEMPERATURA_MINIMA,VELOCIDAD_VIENTO
0,AMAZONAS,LETICIA,2016,248.120000,85.706687,NaN,987.450620,26.177727,49.0,18.8,0.420000
1,AMAZONAS,LETICIA,2017,159.851151,86.607209,2246.66,1000.882407,25.590638,34.8,15.5,0.931521
2,AMAZONAS,LETICIA,2018,163.416194,87.395709,1904.44,1000.432288,24.697919,35.1,16.5,0.904189
3,AMAZONAS,LETICIA,2019,166.521575,89.600315,3177.04,1001.519494,25.892089,45.2,15.8,0.848368
4,AMAZONAS,LETICIA,2020,164.387994,87.991085,2043.42,1001.277717,26.391074,36.8,14.9,0.846891
...,...,...,...,...,...,...,...,...,...,...,...
4399,BOLIVAR,EL GUAMO,2010,NaN,NaN,NaN,NaN,NaN,NaN,28.5,NaN
4401,BOYACA,CHINAVITA,2014,NaN,NaN,NaN,NaN,NaN,NaN,14.6,NaN
4412,CESAR,EL PASO,2007,NaN,NaN,NaN,NaN,NaN,NaN,27.5,NaN
4432,MAGDALENA,SAN SEBASTIAN DE BUENAVISTA,2009,NaN,NaN,NaN,NaN,NaN,NaN,28.4,NaN


**Valores faltantes**

Lidiar con valores faltantes o NaN es una tarea primordial antes de realizar cualquier modelo de aprendizaje de máquina, en este caso encontramos que no todos los municipios presentan las mismas mediciones de variables climáticas, por lo cuál se hace necesario agrupar los datos e imputar los valores nulos.

Se utilizará una imputación por la mediana de cada municipio de cada departamento para todas las variables climáticas, excepto para la temperatura máxima y mínima, que se imputarán con la medición máxima y mínima de cada municipio de cada departamento, esto se debe a que al utilizar otro estimador se pueden presentar datos incoherentes, como por ejemplo, que la temperatura máxima sea menor que la temperatura mínima, y viceversa.

Se eliminan los registros donde luego de la imputación continué el valor nulo presente, ya que esto indica que no existe ninguna medición de la variable climática observada para dicho municipio, por lo que no sería coherente imputadar dicho valor con el de otro municipio diferente.

In [ ]:
Variables_Climaticas_Consolidado['DIRECCION_VIENTO'] = Variables_Climaticas_Consolidado['DIRECCION_VIENTO'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['DIRECCION_VIENTO'].transform('median'))
Variables_Climaticas_Consolidado['HUMEDAD'] = Variables_Climaticas_Consolidado['HUMEDAD'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['HUMEDAD'].transform('median'))
Variables_Climaticas_Consolidado['PRECIPITACION'] =Variables_Climaticas_Consolidado['PRECIPITACION'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['PRECIPITACION'].transform('median'))
Variables_Climaticas_Consolidado['PRESION_ATMOSFERICA'] = Variables_Climaticas_Consolidado['PRESION_ATMOSFERICA'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['PRESION_ATMOSFERICA'].transform('median'))
Variables_Climaticas_Consolidado['TEMPERATURA'] = Variables_Climaticas_Consolidado['TEMPERATURA'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['TEMPERATURA'].transform('median'))
Variables_Climaticas_Consolidado['TEMPERATURA_MAXIMA'] = Variables_Climaticas_Consolidado['TEMPERATURA_MAXIMA'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['TEMPERATURA_MAXIMA'].transform('max'))
Variables_Climaticas_Consolidado['TEMPERATURA_MINIMA'] = Variables_Climaticas_Consolidado['TEMPERATURA_MINIMA'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['TEMPERATURA_MINIMA'].transform('min'))
Variables_Climaticas_Consolidado['VELOCIDAD_VIENTO'] = Variables_Climaticas_Consolidado['VELOCIDAD_VIENTO'].fillna(Variables_Climaticas_Consolidado.groupby(['DEPARTAMENTO','MUNICIPIO'])['VELOCIDAD_VIENTO'].transform('median'))
Variables_Climaticas_Consolidado = Variables_Climaticas_Consolidado.dropna(how='any')
Variables_Climaticas_Consolidado = Variables_Climaticas_Consolidado[~(Variables_Climaticas_Consolidado == 0).any(axis=1)]
Variables_Climaticas_Consolidado

,DEPARTAMENTO,MUNICIPIO,AÑO,DIRECCION_VIENTO,HUMEDAD,PRECIPITACION,PRESION_ATMOSFERICA,TEMPERATURA,TEMPERATURA_MAXIMA,TEMPERATURA_MINIMA,VELOCIDAD_VIENTO
0,AMAZONAS,LETICIA,2016,248.120000,85.706687,2043.420,987.450620,26.177727,49.0,18.8,0.420000
1,AMAZONAS,LETICIA,2017,159.851151,86.607209,2246.660,1000.882407,25.590638,34.8,15.5,0.931521
2,AMAZONAS,LETICIA,2018,163.416194,87.395709,1904.440,1000.432288,24.697919,35.1,16.5,0.904189
3,AMAZONAS,LETICIA,2019,166.521575,89.600315,3177.040,1001.519494,25.892089,45.2,15.8,0.848368
4,AMAZONAS,LETICIA,2020,164.387994,87.991085,2043.420,1001.277717,26.391074,36.8,14.9,0.846891
...,...,...,...,...,...,...,...,...,...,...,...
4203,GUAVIARE,SAN JOSE DEL GUAVIARE,2007,132.467687,84.405190,413.800,989.153414,26.087464,1.8,26.2,0.558934
4387,ANTIOQUIA,BELLO,2010,90.885615,71.378609,115.000,858.283284,22.200049,33.5,24.4,2.286206
4399,BOLIVAR,EL GUAMO,2010,206.285929,75.972800,217.865,1001.592399,29.030767,50.0,28.5,1.285693
4401,BOYACA,CHINAVITA,2014,159.762856,86.008598,1756.700,800.513315,16.567907,30.5,14.6,1.064148


**Evaluaciones Agropecuarias Anuales EVA**

Las Evaluaciones Agropecuarias Anuales comprenden una base histórica de los años 2007 a 2021, relacionada con la producción agrícola Nacional, brinda a los actores del sector agrícola información regional y nacional que permite fortalecer los procesos de producción y comercialización de cultivos. 

La información fue obtenida de Datos Abiertos Colombia, con datos proporcionados por el Ministerio de Agricultura  y Desarrollo Rural de Colombia, de la dependencia Agronet. El conjunto de datos está dispuesto de los años 2007-2018, 2019-2020, 2021; por lo que se realizó el tratamiento de los datos para consolidar dos datasets, uno para el cultivo de café y otro para el cultivo de cacao, en los años 2007 - 2021, en el departamento de Santander.

In [ ]:
EVA_2007_2018 = pd.read_csv('/content/EVA 2007-2018.csv')
EVA_2007_2018=EVA_2007_2018[["DEPARTAMENTO","MUNICIPIO","CULTIVO","AÑO","Área Sembrada\n(ha)","Área Cosechada\n(ha)","Producción\n(t)","Rendimiento\n(t/ha)"]]
EVA_2007_2018 =EVA_2007_2018 .rename(columns={"Área Cosechada\n(ha)":"AREA COSECHADA (HA)","Área Sembrada\n(ha)":"AREA SEMBRADA (HA)","Área Cosechada\n(ha)":"AREA COSECHADA (HA)","Producción\n(t)":"PRODUCCION (T)","Rendimiento\n(t/ha)":"RENDIMIENTO (T/HA)"})

EVA_2019_2020 = pd.read_csv('/content/EVA 2019-2020.csv')
EVA_2019_2020=EVA_2019_2020[["Departamento","Municipio","Cultivo","Año","Area Sembrada (ha)","Area Cosechada (ha)","Producción (t)","Rendimiento (t/ha)"]]
EVA_2019_2020=EVA_2019_2020.rename(columns={"Departamento":"DEPARTAMENTO","Municipio":"MUNICIPIO","Cultivo":"CULTIVO","Año":"AÑO","Area Sembrada (ha)":"AREA SEMBRADA (HA)","Area Cosechada (ha)":"AREA COSECHADA (HA)","Producción (t)":"PRODUCCION (T)","Rendimiento (t/ha)":"RENDIMIENTO (T/HA)"})
EVA_2019_2020=EVA_2019_2020.loc[(EVA_2019_2020['CULTIVO'] == 'Cacao')| (EVA_2019_2020['CULTIVO'] == 'Café')]

EVA_2021 = pd.read_csv('/content/EVA 2021.csv')
EVA_2021=EVA_2021[["Departamento","Municipio","Cultivo","Año","Área sembrada","Área cosechada","Producción","Rendimiento"]]
EVA_2021=EVA_2021.rename(columns={"Departamento":"DEPARTAMENTO","Municipio":"MUNICIPIO","Cultivo":"CULTIVO","Año":"AÑO","Área sembrada":"AREA SEMBRADA (HA)","Área cosechada":"AREA COSECHADA (HA)","Producción":"PRODUCCION (T)","Rendimiento":"RENDIMIENTO (T/HA)"})

seteocompleto = pd.concat([EVA_2007_2018,EVA_2019_2020])
seteocompleto = pd.concat([seteocompleto,EVA_2021])

seteocompleto['RENDIMIENTO CALCULADO (T/HA)']=seteocompleto["PRODUCCION (T)"] / seteocompleto["AREA COSECHADA (HA)"]

seteocompleto['DEPARTAMENTO']=seteocompleto['DEPARTAMENTO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
seteocompleto['MUNICIPIO']=seteocompleto['MUNICIPIO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
seteocompleto['CULTIVO']=seteocompleto['CULTIVO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

seteocompleto.drop(seteocompleto.loc[seteocompleto['PRODUCCION (T)']==0].index, inplace=True)
seteocompleto

,DEPARTAMENTO,MUNICIPIO,CULTIVO,AÑO,AREA SEMBRADA (HA),AREA COSECHADA (HA),PRODUCCION (T),RENDIMIENTO (T/HA),RENDIMIENTO CALCULADO (T/HA)
0,ANTIOQUIA,APARTADO,CACAO,2007,1830.0,1640.0,656.00,0.40,0.400000
1,ANTIOQUIA,TURBO,CACAO,2007,1002.0,760.0,304.00,0.40,0.400000
2,ANTIOQUIA,MACEO,CACAO,2007,950.0,798.0,279.00,0.35,0.349624
3,ANTIOQUIA,YALI,CACAO,2007,580.0,269.0,188.00,0.70,0.698885
4,ANTIOQUIA,NECOCLI,CACAO,2007,523.0,151.0,60.00,0.40,0.397351
...,...,...,...,...,...,...,...,...,...
1193,VAUPES,CARURU,CACAO,2021,121.0,106.0,67.84,0.64,0.640000
1194,VAUPES,TARAIRA,CACAO,2021,7.0,3.5,0.55,0.16,0.157143
1195,VICHADA,LA PRIMAVERA,CACAO,2021,70.0,40.0,28.00,0.70,0.700000
1196,VICHADA,SANTA ROSALIA,CACAO,2021,160.0,160.0,96.00,0.60,0.600000


**Consolidación de Datasets Café y Cacao periodo 2007-2021**

In [ ]:
Cafe_Consolidado = seteocompleto.copy()
Cafe_Consolidado = Cafe_Consolidado.loc[Cafe_Consolidado['CULTIVO'] == 'CAFE'] 
Dataset_Input_Cafe = pd.merge(Variables_Climaticas_Consolidado,Cafe_Consolidado, on=["AÑO","DEPARTAMENTO","MUNICIPIO"], how='inner')
Dataset_Input_Cafe = Dataset_Input_Cafe [["AÑO","DEPARTAMENTO","MUNICIPIO","CULTIVO","DIRECCION_VIENTO","HUMEDAD","PRECIPITACION","PRESION_ATMOSFERICA","TEMPERATURA","TEMPERATURA_MAXIMA","TEMPERATURA_MINIMA","VELOCIDAD_VIENTO","AREA SEMBRADA (HA)","AREA COSECHADA (HA)","PRODUCCION (T)","RENDIMIENTO (T/HA)","RENDIMIENTO CALCULADO (T/HA)"]]
Dataset_Input_Cafe = Dataset_Input_Cafe.sort_values("AÑO",ascending=True)
Dataset_Input_Cafe

,AÑO,DEPARTAMENTO,MUNICIPIO,CULTIVO,DIRECCION_VIENTO,HUMEDAD,PRECIPITACION,PRESION_ATMOSFERICA,TEMPERATURA,TEMPERATURA_MAXIMA,TEMPERATURA_MINIMA,VELOCIDAD_VIENTO,AREA SEMBRADA (HA),AREA COSECHADA (HA),PRODUCCION (T),RENDIMIENTO (T/HA),RENDIMIENTO CALCULADO (T/HA)
403,2007,QUINDIO,ARMENIA,CAFE,7.670278,78.626881,1148.7385,857.385230,21.195415,31.6,13.9,1.381478,2103.00,1820.00,2608.00,1.43,1.432967
242,2007,CUNDINAMARCA,QUEBRADANEGRA,CAFE,7.403245,80.761905,112.7000,835.312277,23.381988,32.5,17.2,1.393026,119.00,106.00,82.00,0.77,0.773585
617,2007,VALLE DEL CAUCA,JAMUNDI,CAFE,8.412669,79.210208,4670.3500,904.974165,23.362950,35.4,15.1,1.518608,1631.00,1620.00,1620.00,1.00,1.000000
197,2007,CUNDINAMARCA,CABRERA,CAFE,7.770544,91.100708,1186.1000,710.850546,10.597636,17.6,0.1,2.399549,18.00,15.00,4.00,0.24,0.266667
365,2007,NORTE DE SANTANDER,CUCUTA,CAFE,8.548039,68.782358,123.7000,597.225629,27.770007,39.2,19.3,1.602590,69.00,67.00,24.00,0.36,0.358209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,2021,SANTANDER,LEBRIJA,CAFE,235.572809,99.637873,307.6800,883.436820,21.278930,30.2,16.3,1.666573,43.73,35.74,48.61,1.36,1.360101
690,2021,MAGDALENA,CIENAGA,CAFE,118.839584,69.955918,49.5000,895.429279,23.042079,32.4,16.9,0.228697,10551.87,9468.36,5586.33,0.59,0.590000
111,2021,CAQUETA,FLORENCIA,CAFE,163.511830,57.181182,481.2000,761.440563,26.740440,36.0,19.2,1.474616,1004.45,807.03,443.87,0.55,0.550004
86,2021,CALDAS,RIOSUCIO,CAFE,151.741297,88.416855,2910.0000,789.188942,16.133682,25.9,10.8,1.630054,2832.14,2275.27,2593.81,1.14,1.140001


In [ ]:
Cacao_Consolidado = seteocompleto.copy()
Cacao_Consolidado = Cacao_Consolidado.loc[Cacao_Consolidado['CULTIVO'] == 'CACAO'] 
Dataset_Input_Cacao = pd.merge(Variables_Climaticas_Consolidado,Cacao_Consolidado, on=["AÑO","DEPARTAMENTO","MUNICIPIO"], how='inner')
Dataset_Input_Cacao = Dataset_Input_Cacao [["AÑO","DEPARTAMENTO","MUNICIPIO","CULTIVO","DIRECCION_VIENTO","HUMEDAD","PRECIPITACION","PRESION_ATMOSFERICA","TEMPERATURA","TEMPERATURA_MAXIMA","TEMPERATURA_MINIMA","VELOCIDAD_VIENTO","AREA SEMBRADA (HA)","AREA COSECHADA (HA)","PRODUCCION (T)","RENDIMIENTO (T/HA)","RENDIMIENTO CALCULADO (T/HA)"]]
Dataset_Input_Cacao = Dataset_Input_Cacao.sort_values("AÑO",ascending=True)
Dataset_Input_Cacao

,AÑO,DEPARTAMENTO,MUNICIPIO,CULTIVO,DIRECCION_VIENTO,HUMEDAD,PRECIPITACION,PRESION_ATMOSFERICA,TEMPERATURA,TEMPERATURA_MAXIMA,TEMPERATURA_MINIMA,VELOCIDAD_VIENTO,AREA SEMBRADA (HA),AREA COSECHADA (HA),PRODUCCION (T),RENDIMIENTO (T/HA),RENDIMIENTO CALCULADO (T/HA)
779,2007,GUAVIARE,SAN JOSE DEL GUAVIARE,CACAO,132.467687,84.405190,413.80,989.153414,26.087464,1.8,26.2,0.558934,75.0,20.0,6.00,0.30,0.300000
696,2007,META,PUERTO LOPEZ,CACAO,151.560416,81.202340,2416.10,990.703754,26.102069,43.9,-15.0,1.054518,30.0,27.0,14.00,0.52,0.518519
452,2007,RISARALDA,MARSELLA,CACAO,8.681276,80.146795,769.85,838.541960,20.666914,30.4,14.3,1.901595,163.0,163.0,157.00,0.96,0.963190
386,2007,NORTE DE SANTANDER,CUCUTA,CACAO,8.548039,68.782358,123.70,597.225629,27.770007,39.2,19.3,1.602590,1660.0,1460.0,710.00,0.49,0.486301
490,2007,SANTANDER,BUCARAMANGA,CACAO,5.966291,77.459933,549.20,906.147325,23.511833,33.7,16.8,2.039302,59.0,30.0,18.00,0.60,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,2021,NORTE DE SANTANDER,TOLEDO,CACAO,152.248276,90.566025,2783.60,793.628565,14.881462,26.8,5.9,1.419069,80.0,70.0,49.00,0.70,0.700000
192,2021,CHOCO,CERTEGUI,CACAO,178.965462,94.421067,3718.10,1006.593367,23.705534,35.4,15.3,1.038744,64.0,6.0,2.28,0.38,0.380000
91,2021,CAQUETA,SOLANO,CACAO,141.126004,2.978026,738.00,987.586085,26.151447,36.3,19.4,1.503208,685.0,635.0,222.25,0.35,0.350000
248,2021,CUNDINAMARCA,GUADUAS,CACAO,177.009694,86.383901,2605.70,828.052849,18.831751,28.7,13.4,1.734578,49.0,44.0,22.00,0.50,0.500000


In [ ]:
Input=Dataset_Input_Cafe.append(Dataset_Input_Cacao)
Input = Input.sort_values("AÑO",ascending=True)
Input.shape

(1555, 17)

**Valores Atípicos**

Con el objetivo de eliminar resultados atípicos que puedan generar confusión al modelo de aprendizaje automático, se parametrizan ciertos criterios como por ejemplo: la temperatura mínima debe ser menor que la temperatura máxima y la temperatura media estar contenida dentro de este rango de valores.

In [ ]:
Input.drop(Input.loc[Input.TEMPERATURA_MAXIMA< Input.TEMPERATURA_MINIMA].index, inplace=True)
Input.drop(Input.loc[Input.TEMPERATURA_MAXIMA< Input.TEMPERATURA].index, inplace=True)
Input.drop(Input.loc[Input.TEMPERATURA_MINIMA> Input.TEMPERATURA].index, inplace=True)
Input.shape

(1497, 17)

**Altura Sobre el Nivel del Mar (ASNM)**

Se utilizan las combinaciones departamento-municipio para obtener la altitud (metros sobre el nivel del mar) de cada municipio, para esto se utilizan herramientas de web scraping,con google maps y el sitio web https://www.municipio.com.co/ 

In [ ]:
Geos = pd.read_csv('/content/Geolocalizaciones.csv')
data = Input.merge(Geos,on =['DEPARTAMENTO','MUNICIPIO'], how ='left')
data = data[["AÑO","ASNM","CULTIVO","DIRECCION_VIENTO","HUMEDAD","PRECIPITACION","PRESION_ATMOSFERICA","TEMPERATURA","TEMPERATURA_MAXIMA","TEMPERATURA_MINIMA","VELOCIDAD_VIENTO","AREA SEMBRADA (HA)","AREA COSECHADA (HA)","PRODUCCION (T)","RENDIMIENTO (T/HA)","RENDIMIENTO CALCULADO (T/HA)"]]
data = data.astype({'ASNM':'float'})
data

,AÑO,ASNM,CULTIVO,DIRECCION_VIENTO,HUMEDAD,PRECIPITACION,PRESION_ATMOSFERICA,TEMPERATURA,TEMPERATURA_MAXIMA,TEMPERATURA_MINIMA,VELOCIDAD_VIENTO,AREA SEMBRADA (HA),AREA COSECHADA (HA),PRODUCCION (T),RENDIMIENTO (T/HA),RENDIMIENTO CALCULADO (T/HA)
0,2007,1386.0,CAFE,7.670278,78.626881,1148.7385,857.385230,21.195415,31.6,13.9,1.381478,2103.00,1820.00,2608.00,1.43,1.432967
1,2007,1183.0,CAFE,4.472613,89.164155,1455.9000,834.351905,18.462581,29.4,12.3,1.708104,436.00,385.00,206.00,0.54,0.535065
2,2007,1527.0,CAFE,8.311749,88.020945,604.4500,824.094556,18.721168,29.2,12.4,0.914488,8040.00,7187.00,11265.00,1.57,1.567413
3,2007,917.0,CAFE,8.993532,79.060808,637.8000,904.119599,23.940029,36.9,-12.8,1.375171,749.00,695.00,300.00,0.43,0.431655
4,2007,178.0,CACAO,151.560416,81.202340,2416.1000,990.703754,26.102069,43.9,-15.0,1.054518,30.00,27.00,14.00,0.52,0.518519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,2021,917.0,CAFE,175.385979,80.789466,379.7000,910.472953,24.280973,38.7,17.0,0.661378,304.48,247.31,269.57,1.09,1.090008
1493,2021,1937.0,CAFE,176.127306,86.604944,2199.2000,814.633781,17.013524,28.2,10.1,1.190225,449.86,352.76,384.51,1.09,1.090005
1494,2021,969.0,CAFE,147.392346,80.958375,1173.4000,842.753089,23.304938,38.2,-14.1,0.721603,435.55,388.63,528.54,1.36,1.360008
1495,2021,3000.0,CAFE,170.740335,79.005156,371.4000,718.353714,11.882184,22.1,2.1,2.283442,71.00,63.00,74.34,1.18,1.180000


In [ ]:
#Dataset Cafe
cafeset =data.loc[(data['CULTIVO']=="CAFE")]
cafeset.to_csv('inputcafe.csv', encoding = 'utf-8-sig') 
files.download('inputcafe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Dataset Cacao
cacaoset =data.loc[(data['CULTIVO']=="CACAO")]
cacaoset.to_csv('inputcacao.csv', encoding = 'utf-8-sig') 
files.download('inputcacao.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>